In [6]:
# Import packages
import pyspark
import dxpy
import dxdata

In [7]:
dxdata.__version__

'0.41.0'

In [8]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [9]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

In [10]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [13]:
dataset.entities

[<Entity "participant">,
 <Entity "covid19_result_england">,
 <Entity "covid19_result_scotland">,
 <Entity "covid19_result_wales">,
 <Entity "gp_clinical">,
 <Entity "gp_scripts">,
 <Entity "gp_registrations">,
 <Entity "hesin">,
 <Entity "hesin_diag">,
 <Entity "hesin_oper">,
 <Entity "hesin_critical">,
 <Entity "hesin_maternity">,
 <Entity "hesin_delivery">,
 <Entity "hesin_psych">,
 <Entity "death">,
 <Entity "death_cause">,
 <Entity "omop_death">,
 <Entity "omop_device_exposure">,
 <Entity "omop_note">,
 <Entity "omop_observation">,
 <Entity "omop_drug_exposure">,
 <Entity "omop_observation_period">,
 <Entity "omop_person">,
 <Entity "omop_procedure_occurrence">,
 <Entity "omop_specimen">,
 <Entity "omop_visit_detail">,
 <Entity "omop_visit_occurrence">,
 <Entity "omop_dose_era">,
 <Entity "omop_drug_era">,
 <Entity "omop_condition_era">,
 <Entity "omop_condition_occurrence">,
 <Entity "omop_measurement">,
 <Entity "olink_instance_0">,
 <Entity "olink_instance_2">,
 <Entity "olink_

In [17]:
participant = dataset['participant']

In [21]:
# Returns all field objects for a given UKB showcase field id

def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given UKB showcase field id

def field_names_for_id(field_id):
    return [f.name for f in fields_for_id(field_id)]

# Returns all field objects for a given title keyword

def fields_by_title_keyword(keyword):
    from distutils.version import LooseVersion
    fields = list(participant.find_fields(lambda f: keyword.lower() in f.title.lower()))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given title keyword

def field_names_by_title_keyword(keyword):
    return [f.name for f in fields_by_title_keyword(keyword)]

# Returns all field titles for a given title keyword

def field_titles_by_title_keyword(keyword):
    return [f.title for f in fields_by_title_keyword(keyword)]

In [18]:
# Participant sex
field_names_for_id('31')

/tmp/ipykernel_166/339987039.py:7: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return sorted(fields, key=lambda f: LooseVersion(f.name))


['p31']

In [19]:
field_names_for_id('21003')

/tmp/ipykernel_166/339987039.py:7: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return sorted(fields, key=lambda f: LooseVersion(f.name))


['p21003_i0', 'p21003_i1', 'p21003_i2', 'p21003_i3']

In [20]:
# Pulse rate has multiple instances and array indices (measured twice in each visit)
field_names_for_id('102')

/tmp/ipykernel_166/339987039.py:7: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return sorted(fields, key=lambda f: LooseVersion(f.name))


['p102_i0_a0',
 'p102_i0_a1',
 'p102_i1_a0',
 'p102_i1_a1',
 'p102_i2_a0',
 'p102_i2_a1',
 'p102_i3_a0',
 'p102_i3_a1']

In [ ]:
field_names = ['eid', 'p31', 'p21022'] \
    + field_names_for_id('41262') \
    + field_names_by_title_keyword('standing height')
field_names

In [23]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

In [29]:
df.filter((df.p21022 >= 50) & (df.p31 == 0)).count()

208593

<font size="12">load wearable data</font>

In [44]:
field_names = ['eid']+field_names_for_id('41270')+field_names_for_id('41271')

/tmp/ipykernel_166/3314766882.py:7: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return sorted(fields, key=lambda f: LooseVersion(f.name))


In [45]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

In [46]:
# See the first five entries as a Pandas DataFrame:
df.limit(5).toPandas()

,eid,p41270,p41271
0,2950871,"[D125, D509, D649, E039, E669, H250, H258, H26...",None
1,4547269,"[E780, F171, G409, H028, H200, I10, I739, I802...",None
2,3509529,"[E119, E831, M169, M1999, M2575, M8612]",[V252]
3,1893827,"[D860, J459, M4792]",None
4,5994000,"[K210, K221, K30, R194]",None
